# Samenvatting

## Association Rules indentificeren
Het identificeren van associatieregels gaat in **drie stappen**:
1. De gegevens in het juiste formaat zetten met datamanagementtechnieken.
2. De **frequenties van de combinaties** berekenen die voorkomen in de transacties.
3. Met de frequenties van deze combinaties kun je daarna **associatieregels** bepalen.

*Association Rules zullen worden beschreven als **AR***

## Stap 1: Datamanagement
Stel:
- Je krijgt van de verkoopafdeling een bestand met verkopen die er als volgt uit kan zien:

| KassaTicket | Product   |
|-------------|-----------|
| 1000123     | Printer   |
| 1000123     | Cartridge |
| 1000123     | Balpen    |
| 1000124     | Papier    |
| 1000124     | Cartridge |
| 1000125     | Papier    |
| 1000125     | Cartridge |
| 1000126     | Printer   |
| 1000126     | Cartridge |
| 1000127     | Printer   |
| 1000127     | Cartridge |
| 1000128     | Balpen    |
| 1000129     | Papier    |
| 1000129     | Cartridge |
| 1000129     | Balpen    |
| 1000130     | Cartridge |
| 1000131     | Printer   |
| 1000131     | Balpen    |
| 1000132     | Papier    |

In [6]:
import pandas as pd

verkopen = pd.DataFrame({
    'KassaTicket': [1000123, 1000123, 1000123, 1000124, 1000124, 1000125, 1000125, 1000126, 1000126, 1000127, 1000127, 1000128, 1000129, 1000129, 1000129, 1000130, 1000131, 1000131, 1000132],
    'Product': ['Printer', 'Cartridge', 'Balpen', 'Papier', 'Cartridge', 'Papier', 'Cartridge', 'Printer', 'Cartridge', 'Printer', 'Cartridge', 'Balpen', 'Papier', 'Cartridge', 'Balpen', 'Cartridge', 'Printer', 'Balpen', 'Papier']
})
verkopen.set_index('KassaTicket', inplace=True)

verkopen

,Product
KassaTicket,
1000123,Printer
1000123,Cartridge
1000123,Balpen
1000124,Papier
1000124,Cartridge
1000125,Papier
1000125,Cartridge
1000126,Printer
1000126,Cartridge


De aankoop van één product komt in dit bestand overeen met één rij in het bestand: als iemand gelijktijdig meerdere dingen koopt krijg je meerdere lijnen.
Het algoritme dat we gaan gebruiken heeft een ander dataformaat nodig: deze heeft één rij voor alle aankopen van één klant nodig.
⇒

In [10]:
transacties = pd.get_dummies(verkopen, columns=['Product'], prefix='',prefix_sep='').groupby(level='KassaTicket').sum()

transacties

,Balpen,Cartridge,Papier,Printer
KassaTicket,,,,
1000123,1,1,0,1
1000124,0,1,1,0
1000125,0,1,1,0
1000126,0,1,0,1
1000127,0,1,0,1
1000128,1,0,0,0
1000129,1,1,1,0
1000130,0,1,0,0
1000131,1,0,0,1


## Stap 2: Frequente patronen identificeren
Met een correcte datastructuur kunnen we op zoek naar de itemsets die vaak samen voorkomen.
Eerst moeten we de relatieve frequenties van alle itemcombinaties berekenen.

In **AR Mining** wordt de relatieve frequentie de **support** genoemd.

### Support

We kunnen de support van 1 item berekenen maar ook van itemsets met meerdere items.

#### Definitie
De support geeft aan hoe vaak een itemset voorkomt in de volledige dataset van **$N\ $** transacties.
De support voor item **$A\ $** kan als volgt worden berekend:
$$support(A) = \frac{\#A}{N}$$

#### Met de hand berekenen
Wanneer je de support van een itemset met de hand wilt berekenen begin je met een lege 'winkelmand' en voeg je telkens één item toe.
Je voegt dan laag per laag een item toe aan de vorige itemset. Zoals hieronder is te zien.
<br>
<img src="rescources/img/Schermafbeelding 2024-03-18 102310.png">

Bovenstaande figuur wordt een **lattice** genoemd. De lattice begint met een lege set ∅ en eindigt met de volledige set van alle items.

Het aantal itemsets dat gevormd kan worden op basis van het aantal items in de winkel kan **astronomisch groot** worden. Het totaal aantal combinaties voor een winkel die $n$ producten verkoopt kan je berekenen met volgende Python code:

In [11]:
from scipy.special import comb

n = 100

sets = sum([comb(n, k) for k in range(1, n+1)])

print('Totaal aantal combinaties voor 100 producten {:.0f}'.format(sets))

Totaal aantal combinaties voor 100 producten 1267650600228240941970748342272


We berekenen de support handmatig voor alle itemsets die we kunnen vormen met de producten uit de transacties.
Dit doen we door te tellen hoe vaak een itemset voorkomt in de transacties en dit te delen door het aantal transacties.

- De support van alle 1 item itemsets:
    - Printer wordt 4 keer verkocht:
        - $support(Printer) = \frac{4}{10} = 0.4$
    - Papier wordt 4 keer verkocht:
        - $support(Papier) = \frac{4}{10} = 0.4$
    - Cartridge wordt 7 keer verkocht:
        - $support(Cartridge) = \frac{7}{10} = 0.7$
    - Balpen wordt 4 keer verkocht:
        - $support(Balpen) = \frac{4}{10} = 0.4$
- De support van alle 2 item itemsets:
    - $support(Printer ∧ Papier) = \frac{0}{10} = 0$